# Retail Banking Challenge 2 - Baseline Submission

This notebook provides a simple baseline for **Retail Banking Challenge 2: Credit Default Prediction**.

**Goal**: Predict `DefaultLabel` (0/1) for each customer-week combination
**Metric**: Macro-F1 - Higher is better

## Instructions:
1. **Replace API credentials** in the first cell with your team's API key and name
2. **Run all cells** to generate and submit baseline predictions
3. **Check the output** for your submission score

This baseline uses only tabular customer panel data with a simple Random Forest classifier.


In [ ]:
# 1. Initialize Client and Load Data

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from agentds import BenchmarkClient

# 🔑 REPLACE WITH YOUR CREDENTIALS
client = BenchmarkClient(
    api_key="your-api-key-here",        # Get from your team dashboard
    team_name="your-team-name-here"     # Your exact team name
)

# Load data from PVC paths
print("📂 Loading Retail Banking Challenge 2 data...")

# Load customer panel data
train_panel = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/customer_panel_train.csv")
test_panel = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/customer_panel_test.csv")

print(f"✅ Data loaded:")
print(f"   Train panel: {train_panel.shape}")
print(f"   Test panel: {test_panel.shape}")
print(f"   Train columns: {list(train_panel.columns)}")
print(f"   Test columns: {list(test_panel.columns)}")


In [ ]:
# 2. Tabular-Only Baseline Model and Predictions

# From data inspection - customer panel columns:
# CustomerID, Week, Utilisation, PaymentRatio, HardInquiries, DefaultLabel (train only)

# Select numeric features for baseline (exclude ID and target columns)
panel_features = ['Week', 'Utilisation', 'PaymentRatio', 'HardInquiries']
print(f"📊 Using panel features: {panel_features}")

# Prepare training data
X_train = train_panel[panel_features].fillna(0).astype(float)
y_train = train_panel['DefaultLabel'].astype(int)  # Target variable

# Prepare test data
X_test = test_panel[panel_features].fillna(0).astype(float)

# Train simple Random Forest classifier baseline
print("🤖 Training Random Forest classifier...")
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Create submission file (format: CustomerID,Week,DefaultLabel)
submission_df = pd.DataFrame({
    'CustomerID': test_panel['CustomerID'],
    'Week': test_panel['Week'],
    'DefaultLabel': predictions
})

# Save predictions
submission_df.to_csv("retailbanking_challenge2_predictions.csv", index=False)
print(f"✅ Predictions saved: {submission_df.shape[0]} predictions")
print(f"   Preview: {submission_df.head(3)}")
print(f"   Default rate: {predictions.mean():.3f} ({predictions.sum()} defaults out of {len(predictions)})")


In [ ]:
# 3. Submit Predictions

# Submit predictions to the competition
print("🚀 Submitting predictions...")

try:
    result = client.submit_prediction("Retailbanking", 2, "retailbanking_challenge2_predictions.csv")
    
    if result['success']:
        print("✅ Submission successful!")
        print(f"   📊 Score: {result['score']:.4f}")
        print(f"   📏 Metric: {result['metric_name']}")
        print(f"   ✔️  Validation: {'Passed' if result['validation_passed'] else 'Failed'}")
    else:
        print("❌ Submission failed!")
        print(f"   Error details: {result.get('details', {}).get('validation_errors', 'Unknown error')}")
        
except Exception as e:
    print(f"💥 Submission error: {e}")
    print("🔧 Check your API key and team name are correct!")

print("\n🎯 Next steps:")
print("   1. Try incorporating relevant information outside this table!")
print("   2. You've completed all Retail Banking challenges!")
